<a href="https://colab.research.google.com/github/jetsonmom/yolov5/blob/main/yolov8_parkingcar_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 내 구글 드라이브에 연동
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# 필요한 패키지와 모듈을 불러옴
import cv2
import numpy as np
import time
import io
import base64
from IPython.display import HTML

In [ ]:
# Detection 하기 전에 원본 동영상을 Display
# video = io.open('gdrive/My Drive/CV/Parking Car Count/video/parking.mp4', 'r+b').read()
video = io.open('/content/gdrive/My Drive/CV/video/Parking Car Count/parking.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video width="30%" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4"/>
             </video>'''.format(encoded.decode('ascii')))

In [ ]:
file_name = '/content/gdrive/My Drive/CV/video/Parking Car Count/parking.mp4'  # Detection 할 원본 동영상
min_confidence = 0.5   # detection 으로 인정할 최소 확률(신뢰도) 지정
output_name = 'parking_car_count_video.mp4'  # Detection 된 output 동영상
elapsed_time = 0       # 총 경과시간 초기화

In [ ]:
def detectAndDisplay(frame):
    start_time = time.time()
    img = cv2.resize(frame, None, fx=0.9, fy=0.9)
    height, width, channels = img.shape

    # YOLOv3의 Detecting model 3가지(320×320, 416×416, 608×608)
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)

    # Showing informations on the screen
    confidences = []    # detection 한 Class 의 신뢰도(확률)를 저장하는 배열 정의
    boxes = []          # detection 한 boxing 정보를 저장하는 배열 정의

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)      # detection 한 Class id
            confidence = scores[class_id]     # detection 한 Class 의 신뢰도(확률)
            # Filter only 'car'
            if class_id == 2 and confidence > min_confidence:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])             # boxing 정보를 boxes 배열에 저장
                confidences.append(float(confidence))  # 신뢰도(확률)을 confidences 배열에 저장

    # apply non-max suppression
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, min_confidence, 0.4)    # 박스안에 박스(노이즈)를 하나로 만들어 준다.
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:    # 노이즈가 제거된 박스만 표시해 준다.
            x, y, w, h = boxes[i]
            label = '{:,.2%}'.format(confidences[i])  # 신뢰도(확률)
            print(i, label)
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 1)
            cv2.rectangle(img, (x, y - 20), (x + w, y), (0, 255, 0), -1)
            cv2.putText(img, label, (x + 5, y - 5), cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 0), 1)

    text = "number of parked cars : {} ".format(len(indexes))
    cv2.putText(img, text, (40, 40), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 0), 10)
    cv2.putText(img, text, (40, 40), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 2)

    process_time = time.time() - start_time
    global elapsed_time
    elapsed_time += process_time   # 총 경과시간 누적
    print("=== A frame took {:.3f} seconds".format(process_time))

    # video 를 disk 에 output 하기 위해 writer 를 초기화한다.
    global writer
    if writer is None and output_name is not None:
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter(output_name, fourcc, 30,
                (img.shape[1], img.shape[0]), True)

    # disk 에 frame 을 write 합니다.
    if writer is not None:
        writer.write(img)

In [ ]:
# Load Yolo
net = cv2.dnn.readNet("gdrive/My Drive/CV/Object Detection_YOLO/yolov3.weights", "gdrive/My Drive/CV/Object Detection_YOLO/yolov3.cfg")
classes = []  # detection 할 Object(Class) list 배열을 정의
with open("gdrive/My Drive/CV/Object Detection_YOLO/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]   # 80개의 Object(class)를 구분할 수 있는 Object의 이름을 classes 배열에 넣어준다.
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# 원본 동영상에서 video stream을 읽어온다.
cap = cv2.VideoCapture(file_name)
writer = None
if not cap.isOpened:
    print('--(!)Error opening video capture')
    exit(0)
while True:
    ret, frame = cap.read()
    if frame is None:
        # close the video file pointers
        cap.release()
        # close the writer point
        writer.release()
        print('--(!) No captured frame -- Break!')
        print("elapsed time {:.3f} seconds".format(elapsed_time))
        break
    detectAndDisplay(frame)